# Projeto de Análise de Erro

Esse projeto busca abordar erros associados a representação computacional de ponto flutuante, na execução de operações de produto interno de vetores N-dimensionais. Para fins de análise, utilizaremos o protudo interno usual definido abaixo:

\begin{equation}
<X, Y> = \sum_{i=1}^{n} x_i y_i = x_1 y_1 + x_2 y_2 + ... + x_n y_n
\end{equation}

Para realizarmos o estudo do impacto do erro de máquina sobre esta operação, desvemos possuir um valor alvo esperado, esse valor ideal representa o resultado desejado ao final da operação, onde a diferença entre o valor real obtido e o valor ideal é o erro gerado durante sua execução. Para obter esse valor ideal, iremos considerar que os vetores nessa análise estão normalizados, isso é, para todo vetor $X$, sua norma é iqual a $1$, logo:

\begin{equation}
||X|| = \sqrt{<X, X>} = <X, X> = 1
\end{equation}

Com o objetivo de definir um escopo mais restrito ao projeto, também iremos considerar que todos os vetores são binários, isso é, para todo vetor $X$, $x_i = 0$ ou $x_i = k$, onde $0 <= i < N$ e $k \in R$. Vale resaltar que este tipo de configuração de vetores é bem comum para representação de dados, como palavras, textos, documentos, taxonomias, classificação, etc.

Este projeto será execultado na linguagem C/C++, perminitindo interpretar mais facilmente os problemas associados aos bits, além disso será ultilizado a representação de ponto flutuante com menor precisão em C/C++, neste caso o "_float_", trata-se de uma representação padronizada pela _IEEE_ com 32 bits nomeada de _single_ (a representação de 16 bits (_half_) não é nativa em C/C++). Por padrão, C/C++ utiliza truncamento na adminstração de dados.

Resumindo, teremos:

> - Operação de produto interno como alvo da ánalise de erro.
> - Todos os vetores que envolvem o problema possuem norma 1, a norma será o valor esperado do produto interno.
> - Todos os vetores são binários, possuindo apenas dois valores possíveis em cada dimensão, reduzindo o escopo do problema.
> - Representação _float_ em C/C++ com 32 bits de precisão com trucamento.

## Procedimentos Iniciais

Para analizar os dados visualmente, iremos importar as bibliotecas gráficas utilizadas no jupyter e no interpretador de C/C++, neste caso, o Xplot da Xeus, que usa como base o bgplot em Python. Também seram iniciados as estruturas de dados padrão que seram utilizadas durante toda a análise.

In [1]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"
#include <iostream>

using namespace std;

float maximum = 0.0f;
vector<double> x;
vector<double> y;

## Implementação Singela de Produto Interno

Como desejamos simular o produto interno para uma situação muito especifica, isso é, para vetores binários normalizados aplicados a relação $<X, X> = 1$, podemos partir do principio de algumas otimizações que não impactam na implementação singela de produto interno, tais como:

> - Produto interno é indiferente quanto a dimensões nulas, podendo ser retiradas da operação.
> - A dimensão do vetor é irrelevante, desde que o número de dimensões não-nulas seja informado.
> - não é necessario a costrução de uma estrutura de array para representação do vetor, por ele ser binário.

In [2]:
float dotNaive(int size, int shift=0){
    float max = 0.0f;
    x = vector<double>(size);
    y = vector<double>(size);
    x[0] = 0; y[0] = 1.0;
    for(int i=1 ; i<size ; i++){
        int aux = i << shift;
        float d = 1.0 / sqrt(aux);
        float sum = 0.0f;
        for(int j=0 ; j<aux ; j++){
            sum += d*d;
        }
        if(max < sum){
            max = sum;
        }
        x[i] = aux;
        y[i] = sum;
    }
    return max;
}

## Primeiro teste de execução

Iremos executar o código para gerar o resultado dos 10000 primeiros vetores com dimensão não-nula até 10000, o resultado segue no gráfico abaixo:

In [7]:
dotNaive(10000);
xpl::linear_scale sx1, sy1;
auto fig1 = xpl::figure_generator().padding_x(0.025).padding_y(0.025).finalize();
fig1.add_mark(xpl::scatter_generator(sx1, sy1).x(x).y(y).default_size(1).finalize());
fig1.add_axis(xpl::axis_generator(sy1).label("Valor Obtido").orientation("vertical").side("left").finalize());
fig1.add_axis(xpl::axis_generator(sx1).label("Número de Dimensões Não-Nulas").finalize());
fig1

A Jupyter widget

## Análise de resultados 

Do gráfico acima podemos observar que ocorreu grande dispersão do valor com relação ao valor esperado, observe também que o erro associado cresce significativamente conforme o número de dimensões não-nulas crescem, demostrando uma forte correlação desta caracteristica com o erro. Observe tambem que o erro parece ser mais intenso em duas regioes especificas demostrando uma configuração "bimodal", sendo menos intenso com valores proximos ao ideial, e menos recorrente nos erros extremos, contudo elevado nos valores de erro medio absoluto. Observe também que o gráfico aparenta demostrar determinados padrões de frequência, como pequenas senoídes, irremos ataca esse padrão de forma a evidencia-lo e demostrar suas características, para isso iremos aplica o produto vetorial em vetores com potencias de 2 elementos não-nulos afim de reforçar a ocorrência do padrão. 

In [4]:
dotNaive(10000, 8);
xpl::linear_scale sx2, sy2;
auto fig2 = xpl::figure_generator().padding_x(0.025).padding_y(0.025).finalize();
fig2.add_mark(xpl::scatter_generator(sx2, sy2).x(x).y(y).default_size(1).finalize());
fig2.add_axis(xpl::axis_generator(sy2).label("Valor Obtido").orientation("vertical").side("left").finalize());
fig2.add_axis(xpl::axis_generator(sx2).label("Número de Dimensões Não-Nulas").finalize());
fig2

A Jupyter widget

In [ ]:
dotNaive(200, 23);
xpl::linear_scale sx3, sy3;
auto fig3 = xpl::figure_generator().padding_x(0.025).padding_y(0.025).finalize();
fig3.add_mark(xpl::scatter_generator(sx3, sy3).x(x).y(y).default_size(1).finalize());
fig3.add_axis(xpl::axis_generator(sy3).label("Valor Obtido").orientation("vertical").side("left").finalize());
fig3.add_axis(xpl::axis_generator(sx3).label("Número de Dimensões Não-Nulas").finalize());
fig3

Observe que os problemas aparentam demostrar certo padrão, ficando mais evindente quanto colocamos o valor da manquisa (23 bits) como valor do passo, fazendo com que os problemas recaiam sobre o expoente (8 bits). Contudo devemos observar essa observação com cautela, pois ele pode evidência problemas de subamostragem, intuido padrãoes que podem não necessariamente existir.

## Estudo da distribuição do erro
Iremos contruir o histograma sobre o erro absoluto obtido a fim de identificar o comportamento assocido a ele

In [3]:
maximum = dotNaive(10000);
xpl::linear_scale sx4, sy4;
xpl::hist hist(sx4, sy4);
for(int i=0 ; i<y.size() ; i++){
    y[i] = abs(1 - y[i]);
}
hist.sample = y;
xpl::figure fig4;
fig4.add_mark(hist);
xpl::axis hx(sx4), hy(sy4);
hy.orientation = "vertical";
fig4.add_axis(hx);
fig4.add_axis(hy);
hist.colors = std::vector<std::string>{"21c0fc"};
fig4

A Jupyter widget

Como podemos observar o erro apresenta comportamento exponencial. Contudo, devemos resaltar que esse resultado tambem pode esta tendendioso por consequência da subamostragem que foi execultada (10000 vetores).

## Causas do problema
O problema acontece principalmente por dois motivos:
> - O primeiro motivo são os erros proveniente da normalização numérica, gerando problemas com arredondamento / truncamento na representação. Casos de dizimas periódicas que podem ser arredondadas são bons exemplos desse erro ($1/3 = 0.3333334$).
> - O segundo caso é um erro de operação gerado pelo somatório sequêncial. Realizamos duas operação ao calcular o produto interno de um vetor, isso é, a multiplicação de um elemento por ele mesmo (potência de 2) e a soma de todos essas potências. Como sabemos que o vetor é normalizado, teremos que os valores dos elementos variam no intervalo $[1, 0]$, isso faz com que a potência de 2 de um dado elemento no valor $x$ produza um número menor ou igual a $x$, contudo esses valores tendem as ser muito pequenos, pois normalmente $x <<< 1$, fazendo com que $x^2 <<< x$. Até então isso não geraria problema algum, se não fosse a soma sequêncial de valores, a cada potência que realizamos, somamos o valor obtido em um valor total $t$ que retorna a resposta do somatório, contudo, conforme $t$ tende ao valor de $1$ da norma, o valor de $x^2$ deixa de impactar na soma por $x^2 <<< 1$, causando o surgimento de erros de operação. Isso é, o valor do resultado da potência, deixa de fazer impacto sobre a soma total.

![title](imagens/lista.png)

## Solução para minimização do erro
Dentre os problemas que podemos atacar, iremos resaltar o segundo, pois os erros ligados a arredondamento / truncamento de dados, tendem a soluções com representações complexa de estruturas, que não desejamos atacar no contexto desse projeto, nesse caso desejamos minimizar os erros ligados a operação de soma sequêncial. Sendo assim, precisamos garantir que valores pequenos não sejam desprezados com relação a soma de valores grandes, isso pode ser contornado com o uso de uma soma distribuida, que realiza pequenos somatórios somatórios, e posteriormente, acumula em somatórios maiores como pode ser descrito na árvore abaixo:

![title](imagens/arvore.png)

In [4]:
float sumDist(float num, int bgn, int end){
    if(bgn == end-1){
        return num;
    }
    int hlf = (bgn + end)/2;
    return sumDist(num, bgn, hlf) + sumDist(num, hlf, end);
}

In [5]:
float dot(int size, int shift=0){
    float max = 0.0f;
    x = vector<double>(size);
    y = vector<double>(size);
    x[0] = 0; y[0] = 1.0;
    for(int i=1 ; i<size ; i++){
        int aux = i << shift;
        float d = 1.0 / sqrt(aux);
        float sum = sumDist(d*d, 0, i);
        if(max < sum){
            max = sum;
        }
        x[i] = aux;
        y[i] = sum;
    }
    return max;
}

In [6]:
dot(10000);
xpl::linear_scale sx5, sy5;
auto fig5 = xpl::figure_generator().padding_x(0.025).padding_y(0.025).finalize();
fig5.add_mark(xpl::scatter_generator(sx5, sy5).x(x).y(y).default_size(1).finalize());
fig5.add_axis(xpl::axis_generator(sy5).label("Valor Obtido").orientation("vertical").side("left").finalize());
fig5.add_axis(xpl::axis_generator(sx5).label("Número de Dimensões Não-Nulas").finalize());
fig5

A Jupyter widget

Como podemos observar, o erro ainda existe, contudo nesta nova abordagem, visualizamos apenas erros provenientas da operação de normalização, se tornando mais regular. 

In [7]:
xpl::linear_scale sx7, sy7;
xpl::hist hist2(sx7, sy7);
for(int i=0 ; i<y.size() ; i++){
    y[i] = abs(1 - y[i]);
}
hist2.sample = y;
xpl::figure fig7;
fig7.add_mark(hist2);
xpl::axis hx2(sx4), hy2(sy4);
hy2.orientation = "vertical";
fig7.add_axis(hx2);
fig7.add_axis(hy2);
hist2.colors = std::vector<std::string>{"21c0fc"};
fig7

A Jupyter widget

A regularidade do erro se torna ainda mais evidente na visualização do histograma acima, que apresenta uma distribuição muito especifica com relação ao problema.

In [8]:
dot(10000);
y[y.size()-1] = maximum;
y[y.size()-2] = 2.0f - maximum;
xpl::linear_scale sx6, sy6;
auto fig6 = xpl::figure_generator().padding_x(0.025).padding_y(0.025).finalize();
fig6.add_mark(xpl::scatter_generator(sx6, sy6).x(x).y(y).default_size(1).finalize());
fig6.add_axis(xpl::axis_generator(sy6).label("Valor Obtido").orientation("vertical").side("left").finalize());
fig6.add_axis(xpl::axis_generator(sx6).label("Número de Dimensões Não-Nulas").finalize());
fig6

A Jupyter widget

Se observamos a disposisão dos dados sobre a mesma escala, podemos analizar que ocorreu redução significativa do erro com relação a abordagem singela 